## comp_mean_grid
Compute grid-related variables from time-average data (SSH), including corrections for vertical grid metrics

Namely:
* corrected e3t and e3w
* hbot (bottom depth)


In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

from pathlib import Path
import os

import xarray as xr
from xorca.lib import load_xorca_dataset
import itidenatl.gridop as gop


In [2]:
from dask.distributed import Client, LocalCluster, wait
cluster = LocalCluster(threads_per_worker=7) #n_workers=24, threads_per_worker=1, memory_limit=8e6,silence_logs=50
client = Client(cluster)
client

/scratch/cnt0024/ige2071/nlahaye/conda38/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38065 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:37879 Dashboard: http://127.0.0.1:38065/status,Cluster Workers: 8 Cores: 56 Memory: 251.63 GiB


In [3]:
scratch = Path(os.getenv("SCRATCHDIR"))
workdir = Path("/work/CT1/ige2071/nlahaye")
grid_path = scratch #Path("/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/")
mean_path = Path("/work/CT1/ige2071/SHARED/mean")

ssh_fname = "global_mean_gridT-2D.zarr"
grid_fname = "mesh_mask_eNATL60_3.6.nc"

out_file = scratch/"eNATL60_mean_grid_z.zarr"

grid_files = [grid_path/grid_fname]

In [4]:
%%time
chk_xy, chk_z = 1200, 10
chunks = {dim:chk_xy for dim in ["x_c", "x_r", "y_c", "y_r"]}
chunks.update({dim:chk_z for dim in ["z_c", "z_l"]})

ds_tot = load_xorca_dataset(data_files=[], aux_files=grid_files,
                              decode_cf=True, model_config="nest", target_ds_chunks=chunks
                             )
print("initial dataset is {:.1f} GB".format(ds_tot.nbytes/1e9))
ds_new = ds_tot.drop_dims(("y_r", "x_r"))\
                .reset_coords(names=("depth_c_3d", "depth_l_3d", "e1t", "e2t"), drop=True)
print("working dataset is {:.1f} GB".format(ds_new.nbytes/1e9))


initial dataset is 525.4 GB
working dataset is 201.8 GB
CPU times: user 15.6 s, sys: 1.22 s, total: 16.8 s
Wall time: 16.5 s


<xarray.Dataset>
Dimensions:    (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
  * z_c        (z_c) int64 1 2 3 4 5 6 7 8 9 ... 293 294 295 296 297 298 299 300
  * z_l        (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
  * y_c        (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
  * x_c        (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
    depth_c    (z_c) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth_l    (z_l) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    llat_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    llon_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    e3t        (z_c, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3w        (z_l, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmask      (z_c, y_c, x_c) int8 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmaskutil  (y_c, x_c) int8 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
Data variables:
    *empty*

In [5]:
varnames = [var.name for var in ds_new.coords.values() if len(var.dims)==3]
ds_new = ds_new.reset_coords(varnames)
ds_new

<xarray.Dataset>
Dimensions:    (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
  * z_c        (z_c) int64 1 2 3 4 5 6 7 8 9 ... 293 294 295 296 297 298 299 300
  * z_l        (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
  * y_c        (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
  * x_c        (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
    depth_c    (z_c) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth_l    (z_l) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    llat_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    llon_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    tmaskutil  (y_c, x_c) int8 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
Data variables:
    e3t        (z_c, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3w        (z_l, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmask      (z_c, y_c, x_c) int8 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>

In [14]:
%%time
### store
lavar = varnames[0]
ds_new.get([lavar]).to_zarr(out_file, mode="w")

CPU times: user 8min 44s, sys: 15 s, total: 8min 59s
Wall time: 9min 2s


In [19]:
client.restart()

Client Scheduler: tcp://127.0.0.1:41710 Dashboard: http://127.0.0.1:44518/status,Cluster Workers: 8 Cores: 56 Memory: 251.63 GiB


In [20]:
%%time
for lavar in varnames[1:]:
    ds_new.get([lavar]).to_zarr(out_file, mode="a")
    print("done with", lavar)

done with e3w
done with tmask
CPU times: user 17min 17s, sys: 32.4 s, total: 17min 49s
Wall time: 17min 58s


In [25]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:41710 Dashboard: http://127.0.0.1:44518/status,Cluster Workers: 8 Cores: 56 Memory: 251.63 GiB


In [26]:
### load and merge SSH, compute hbot
ds_re = xr.open_zarr(out_file)

dims = {"ssh": ("x_c", "y_c"), "temp":("x_c", "y_c", "z_c"), "salt":("x_c", "y_c", "z_c")}
dimap = {"x":"x_c", "y":"y_c", "deptht":"z_c"}
chk_tot = ds_re.chunks

def get_var(path, varname):
    da = xr.open_zarr(path)[varname].squeeze()
    da = da.rename({dim:dimap[dim] for dim in da.dims})
    da = da.chunk({dim:chk_tot[dim] for dim in da.dims})
    return da

ds_re["sossheig"] = get_var(mean_path/ssh_fname, "sossheig").persist()

ds_re.sossheig.encoding.pop("chunks") # otherwise writing zarr raise error

ds_re

<xarray.Dataset>
Dimensions:    (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
  * x_c        (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
  * y_c        (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
    depth_c    (z_c) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth_l    (z_l) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    llat_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    llon_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    tmaskutil  (y_c, x_c) int8 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
  * z_c        (z_c) int64 1 2 3 4 5 6 7 8 9 ... 293 294 295 296 297 298 299 300
  * z_l        (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
Data variables:
    e3t        (z_c, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3w        (z_l, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmask      (z_c, y_c, x_c) int8 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    sossheig   (y_c, x_c) float64 dask.array<chunksize=(1200, 1200), meta=np.ndarray>

In [27]:
%%time
### compute hbot
ds_re["hbot"] = gop.get_hbot(ds_re).where(ds_re.tmaskutil).persist()
ds_re

CPU times: user 290 ms, sys: 4.96 ms, total: 295 ms
Wall time: 289 ms


<xarray.Dataset>
Dimensions:    (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
  * x_c        (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
  * y_c        (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
    depth_c    (z_c) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth_l    (z_l) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    llat_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    llon_cc    (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    tmaskutil  (y_c, x_c) int8 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
  * z_c        (z_c) int64 1 2 3 4 5 6 7 8 9 ... 293 294 295 296 297 298 299 300
  * z_l        (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
Data variables:
    e3t        (z_c, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3w        (z_l, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmask      (z_c, y_c, x_c) int8 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    sossheig   (y_c, x_c) float64 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    hbot       (y_c, x_c) float64 dask.array<chunksize=(1200, 1200), meta=np.ndarray>

In [28]:
%%time
### store result
ds_re.get(["sossheig", "hbot"]).to_zarr(out_file, mode="a")

CPU times: user 334 ms, sys: 87.5 ms, total: 422 ms
Wall time: 880 ms


In [29]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:41710 Dashboard: http://127.0.0.1:44518/status,Cluster Workers: 8 Cores: 56 Memory: 251.63 GiB


In [6]:
# compute correction for e3t and e3w
# cf gridop.get_rec_e3z

if True:
    ds_re = xr.open_zarr(out_file)
    
lacorr = (1. + ds_re.sossheig / ds_re.hbot).persist()
#(ds_re["e3t"] * lacorr).where(ds_new.tmaskutil)
#ds_new["e3w"] = (ds_new["e3w"] * lacorr).where(ds_new.tmaskutil)


In [8]:
%%time
### compute corrected vertical grid metrics and store in zarr
(ds_re.get(["e3t", "e3w"]) * lacorr).where(ds_re.tmaskutil).to_zarr(out_file, mode="a")

CPU times: user 20.8 s, sys: 2.66 s, total: 23.4 s
Wall time: 1min 4s
